In [1]:
from podpac.core.coordinates import Coordinates, clinspace, UniformCoordinates1d, StackedCoordinates
from podpac.core.data import interpolate
from podpac.core.data.interpolate import (Interpolation, InterpolationException,
                                          Interpolator, INTERPOLATION_METHODS,
                                          INTERPOLATION_SHORTCUTS, NearestNeighbor)
from podpac.core.data.datasource import DataSource
from podpac.core.units import UnitsDataArray

import numpy as np

In [16]:
reqcoords = Coordinates([[-.5, 1.5, 3.5], [.5, 2.5, 4.5]], dims=['lat', 'lon'])
srccoords = Coordinates([[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]], dims=['lat', 'lon'])

interp = Interpolation('nearest_preview')

srccoords, srccoords_index = srccoords.intersect(reqcoords, outer=True, return_indices=True)
coords, cidx = interp.select_coordinates(reqcoords, srccoords, srccoords_index)

In [29]:
# test when selection is applied serially
reqcoords = Coordinates([[-.5, 1.5, 3.5], [.5, 2.5, 4.5]], dims=['lat', 'lon'])
srccoords = Coordinates([[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]], dims=['lat', 'lon'])

interp = Interpolation({
    'lat': 'nearest_preview',
    'lon': 'nearest_preview'
})

srccoords, srccoords_index = srccoords.intersect(reqcoords, outer=True, return_indices=True)
coords, cidx = interp.select_coordinates(reqcoords, srccoords, srccoords_index)

In [31]:
reqcoords = Coordinates([[-.5, 1.5, 3.5], [.5, 2.5, 4.5]], dims=['lat', 'lon'])
srccoords = Coordinates([([0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5])], dims=['lat_lon'])
srccoords, srccoords_index = srccoords.intersect(reqcoords, outer=True, return_indices=True)

for src_dim, idx in zip(srccoords, srccoords_index):
    print(src_dim)
    print(idx)

lat_lon
slice(0, 5, None)


In [30]:
len(reqcoords['lat'])
np.all(coords['lat'].coordinates == np.array([0, 2, 4]))

True

In [5]:
class TestInterp(Interpolator):
    dims_supported = ['lat', 'lon']
    def interpolate(self, udims, source_coordinates, source_data, requested_coordinates, output_data):
        output_data = source_data
        return source_coordinates, source_data, output_data

# test if source and data are both length one
reqcoords = Coordinates([[-.5, 1.5, 3.5], [.5, 2.5, 4.5]], dims=['lat', 'lon'])
srccoords = Coordinates([[0, 2, 4], [0, 3, 4]], dims=['lat', 'lon'])
srcdata = UnitsDataArray(np.random.rand(3, 3),
                         coords=[srccoords[c].coordinates for c in srccoords],
                         dims=srccoords.dims)
outdata = UnitsDataArray(np.zeros(srcdata.shape),
                         coords=[reqcoords[c].coordinates for c in reqcoords],
                         dims=reqcoords.dims)

interp = Interpolation({('lat', 'lon'): {'method': 'test', 'interpolators': [TestInterp]}})

outdata = interp.interpolate(srccoords, srcdata, reqcoords, outdata)

In [8]:
np.all(outdata == srcdata)

True

In [3]:
# source_coordinates = Coordinates([clinspace(-25, 25, 51), clinspace(-25, 25, 51)], dims=['lat', 'lon'])
source_coordinates = Coordinates([(np.linspace(-25, 25, 51), np.linspace(-25, 25, 51))], dims=['lat_lon'])
requested_coordinates = Coordinates([clinspace(-15, 15, 51), clinspace(-15, 15, 51)], dims=['lat', 'lon'])

source_coordinates, source_coordinates_index = source_coordinates.intersect(requested_coordinates, outer=True, return_indices=True)

new_coords =[]
new_coords_idx = []
for dim, idx in zip(source_coordinates, source_coordinates_index):
    if isinstance(source_coordinates[dim], StackedCoordinates):
        
    
    src_coords = source_coordinates[dim]
    dst_coords = requested_coordinates[dim]

    if isinstance(dst_coords, UniformCoordinates1d):
        dst_start = dst_coords.start
        dst_stop = dst_coords.stop
        dst_delta = dst_coords.step
    else:
        dst_start = dst_coords.coordinates[0]
        dst_stop = dst_coords.coordinates[-1]
        dst_delta = (dst_stop-dst_start) / (dst_coords.size - 1)

    if isinstance(src_coords, UniformCoordinates1d):
        src_start = src_coords.start
        src_stop = src_coords.stop
        src_delta = src_coords.step
    else:
        src_start = src_coords.coordinates[0]
        src_stop = src_coords.coordinates[-1]
        src_delta = (src_stop-src_start) / (src_coords.size - 1)

    ndelta = max(1, np.round(dst_delta / src_delta))

    c = UniformCoordinates1d(src_start, src_stop, ndelta*src_delta, **src_coords.properties)

    if isinstance(idx, slice):
        idx = slice(idx.start, idx.stop, int(ndelta))
    else:
        idx = slice(idx[0], idx[-1], int(ndelta))

    new_coords.append(c)
    new_coords_idx.append(idx)

lat_lon
slice(9, 42, None)


KeyError: "dimension 'lat_lon' not found in Coordinates ('lat', 'lon')"

In [11]:
isinstance(source_coordinates['lat_lon'], StackedCoordinates)

True

In [12]:
[c.intersect(requested_coordinates, outer=True, return_indices=True)[1] for c in stacked._coords]

slice(9, 42, None)
[-16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.
  -2.  -1.   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.
  12.  13.  14.  15.  16.]
ArrayCoordinates1d(lat): Bounds[-16.0, 16.0], N[33], ctype['midpoint']
slice(9, 42, None)
[-16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.
  -2.  -1.   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.
  12.  13.  14.  15.  16.]
ArrayCoordinates1d(lon): Bounds[-16.0, 16.0], N[33], ctype['midpoint']


[slice(9, 42, None), slice(9, 42, None)]

In [6]:
interp = Interpolation({('lat',): 'nearest', 'lon': 'bilinear'}, COORDINATES)

In [2]:
# create a few dummy interpolators that handle certain dimensions
# (can_select is defined by default to look at dims_supported)
class TimeLat(Interpolator):
    dims_supported=['time', 'lat']

class LatLon(Interpolator):
    dims_supported=['lat', 'lon']
    
class Lon(Interpolator):
    dims_supported=['lon']


# set up a strange interpolation definition
# we want to interpolate (lat, lon) first, then after (time, alt)
interp = Interpolation({
    ('lat', 'lon'): {
        'method': 'myinterp',
        'interpolators': [LatLon, TimeLat]
    },
    ('time', 'alt'): {
        'method': 'myinterp',
        'interpolators': [TimeLat, Lon]
    }
})

reqcoords = Coordinates([[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]], dims=['lat', 'lon', 'time', 'alt'])
srccoords = Coordinates([[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]], dims=['lat', 'lon', 'time', 'alt'])
interpolator_queue = interp._select_interpolator_queue(reqcoords, srccoords, 'can_select', strict=True)

InterpolationException: Requested dimensions ['alt', 'time'] can't be handled by interpolation definition:
 Interpolation
	('lat', 'lon'): myinterp, ['LatLon', 'TimeLat'], {}
	('time', 'alt'): myinterp, ['TimeLat', 'Lon'], {}
	('default',): nearest, ['NearestNeighbor'], {}